In [1]:
import pandas as pd
from pandas.io.gbq import read_gbq
import numpy as np

project = "spheric-crow-161317"
sample_query = "SELECT * FROM `bigquery-public-data.new_york.311_service_requests` WHERE complaint_type LIKE '%Noise%' LIMIT 1000"
df = read_gbq(query=sample_query, project_id=project, dialect='standard')
df.head()

/Users/moorissatjokro/anaconda/lib/python3.6/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access bigquery_credentials.dat: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))



Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/v2/auth?client_id=495642085510-k0tmvj2m941jhre2nbqka17vqpjfddtd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&access_type=offline&response_type=code

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver

Authentication successful.
Requesting query... ok.
Query running...
Query done.
Processed: 8.8 Gb

Retrieving results...
Got 1000 rows.

Total time taken 4.18 s.
Finished at 2017-03-25 11:24:04.


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,garage_lot_name,ferry_direction,ferry_terminal_name,latitude,longitude,location
0,33603370,2016-06-15 11:13:05,2016-11-07 11:53:50,EDC,Economic Development Corporation,Noise - Helicopter,Other,Above Address,10023,315 WEST 70 STREET,...,,,,,,,,40.778748,-73.986073,"(40.778747935881874, -73.9860734644156)"
1,35544801,2017-02-21 09:29:00,2017-02-21 12:00:00,DSNY,Manhattan 08,Collection Truck Noise,21 Collection Truck Noise,Sidewalk,10075,301 EAST 79 STREET,...,,,,,,,,40.773193,-73.954693,"(40.7731927560976, -73.95469288994452)"
2,20374419,2011-05-06 12:53:52,2012-03-14 14:48:30,EDC,Economic Development Corporation,Noise - Helicopter,Other,Above Address,11231,,...,,,,,,,,40.677329,-74.012928,"(40.67732932538408, -74.01292834062261)"
3,16516956,2010-04-24 03:06:00,2010-04-30 03:45:00,DEP,Department of Environmental Protection,Noise,Noise: Jack Hammering (NC2),,,,...,,,,,,,,NaN,NaN,
4,18265155,2010-07-14 01:04:00,NaT,DEP,Department of Environmental Protection,Noise,Noise: Private Carting Noise (NQ1),,,,...,,,,,,,,NaN,NaN,


In [2]:
pd.__version__

'0.19.2'

In [3]:
import geopandas as gpd

In [4]:
!pip install fiona shapely geopandas

In [8]:
# http://wrobstory.github.io/2013/10/mapping-data-python.html

In [25]:
# read data
df = pd.read_csv("NYPD_Complaint_Data_Current_YTD.csv")
df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,586068632,12/31/2016,23:55:00,12/31/2016,23:56:00,12/31/2016,235,DANGEROUS DRUGS,511.0,"CONTROLLED SUBSTANCE, POSSESSI",...,14,INSIDE,BUS TERMINAL,NaN,NaN,986882.0,214802.0,40.756266,-73.990501,"(40.756266207, -73.990501248)"
1,155423129,12/31/2016,23:40:00,12/31/2016,23:50:00,12/31/2016,105,ROBBERY,389.0,"ROBBERY,DWELLING",...,43,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,1021170.0,241240.0,40.828755,-73.866594,"(40.828754623, -73.866593516)"
2,653964645,12/31/2016,23:30:00,12/31/2016,23:31:00,12/31/2016,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,25,FRONT OF,STREET,NaN,NaN,1001512.0,234334.0,40.809860,-73.937644,"(40.809859893, -73.937644103)"
3,988275798,12/31/2016,23:25:00,NaN,NaN,12/31/2016,235,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",...,7,OPPOSITE OF,STREET,NaN,NaN,987182.0,201484.0,40.719711,-73.989424,"(40.719711494, -73.9894242)"
4,225104473,12/31/2016,23:24:00,12/31/2016,23:30:00,12/31/2016,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",...,102,NaN,STREET,NaN,NaN,1026086.0,192340.0,40.694515,-73.849134,"(40.694514975, -73.849134227)"


In [26]:
df.shape

(478804, 24)

In [27]:
df[['Latitude','Longitude']].head()

,Latitude,Longitude
0,40.756266,-73.990501
1,40.828755,-73.866594
2,40.809860,-73.937644
3,40.719711,-73.989424
4,40.694515,-73.849134


In [28]:
import shapely, geopandas, matplotlib
#boros = geopandas.GeoDataFrame.from_file('nyct2010_13a/nyct2010.shp')
boros = geopandas.GeoDataFrame.from_file('geo_export_be3a4049-303e-42f1-a875-70e07e3011a3.shp')

In [48]:
#turn lat long into shapely points
df.dropna(axis=0, subset=['Latitude','Longitude'], inplace=True)
df['lonlat'] = list(zip(df.Longitude, df.Latitude))

In [49]:
df.lonlat.values[0]

(-73.990501248000001, 40.756266206999996)

In [50]:
df['geometry'] = df[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))

In [51]:
df

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,lonlat,geometry
0,586068632,12/31/2016,23:55:00,12/31/2016,23:56:00,12/31/2016,235,DANGEROUS DRUGS,511.0,"CONTROLLED SUBSTANCE, POSSESSI",...,BUS TERMINAL,NaN,NaN,986882.0,214802.0,40.756266,-73.990501,"(40.756266207, -73.990501248)","(-73.990501248, 40.756266207)",POINT (-73.990501248 40.756266207)
1,155423129,12/31/2016,23:40:00,12/31/2016,23:50:00,12/31/2016,105,ROBBERY,389.0,"ROBBERY,DWELLING",...,RESIDENCE - APT. HOUSE,NaN,NaN,1021170.0,241240.0,40.828755,-73.866594,"(40.828754623, -73.866593516)","(-73.866593516, 40.828754623)",POINT (-73.86659351600001 40.828754623)
2,653964645,12/31/2016,23:30:00,12/31/2016,23:31:00,12/31/2016,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,STREET,NaN,NaN,1001512.0,234334.0,40.809860,-73.937644,"(40.809859893, -73.937644103)","(-73.937644103, 40.809859893)",POINT (-73.937644103 40.809859893)
3,988275798,12/31/2016,23:25:00,NaN,NaN,12/31/2016,235,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",...,STREET,NaN,NaN,987182.0,201484.0,40.719711,-73.989424,"(40.719711494, -73.9894242)","(-73.9894242, 40.719711494)",POINT (-73.9894242 40.719711494)
4,225104473,12/31/2016,23:24:00,12/31/2016,23:30:00,12/31/2016,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",...,STREET,NaN,NaN,1026086.0,192340.0,40.694515,-73.849134,"(40.694514975, -73.849134227)","(-73.849134227, 40.694514975)",POINT (-73.84913422699999 40.694514975)
5,428909890,12/31/2016,23:20:00,12/31/2016,23:25:00,12/31/2016,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",...,RESIDENCE - APT. HOUSE,NaN,NaN,995108.0,175859.0,40.649371,-73.960872,"(40.649370541, -73.960872294)","(-73.960872294, 40.649370541)",POINT (-73.960872294 40.649370541)
6,313457048,12/31/2016,23:20:00,12/31/2016,23:25:00,12/31/2016,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,RESIDENCE - APT. HOUSE,NaN,NaN,998611.0,187749.0,40.682001,-73.948223,"(40.682000963, -73.948223153)","(-73.948223153, 40.682000963)",POINT (-73.948223153 40.682000963)
7,816766111,12/31/2016,23:19:00,NaN,NaN,12/31/2016,126,MISCELLANEOUS PENAL LAW,198.0,CRIMINAL CONTEMPT 1,...,STREET,NaN,NaN,989083.0,205732.0,40.731370,-73.982563,"(40.73137039, -73.982563257)","(-73.982563257, 40.73137039)",POINT (-73.982563257 40.73137039)
8,323812425,12/31/2016,23:15:00,12/31/2016,23:35:00,12/31/2016,348,VEHICLE AND TRAFFIC LAWS,916.0,LEAVING SCENE-ACCIDENT-PERSONA,...,STREET,NaN,NaN,999341.0,172861.0,40.641135,-73.945624,"(40.641135477, -73.945624473)","(-73.945624473, 40.641135477)",POINT (-73.945624473 40.641135477)
9,783505463,12/31/2016,23:14:00,12/31/2016,23:14:00,12/31/2016,344,ASSAULT 3 & RELATED OFFENSES,113.0,"MENACING,UNCLASSIFIED",...,RESIDENCE-HOUSE,NaN,NaN,1042426.0,187538.0,40.681242,-73.790251,"(40.681242296, -73.790251266)","(-73.790251266, 40.681242296)",POINT (-73.790251266 40.681242296)


In [52]:
noisegdf = gpd.GeoDataFrame(df)
noisegdf.crs = {'init': 'epsg:4326'} ## noisegdf did not have a defined crs, but it is clearly in 4326: lat/lon
noisegdf.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,lonlat,geometry
0,586068632,12/31/2016,23:55:00,12/31/2016,23:56:00,12/31/2016,235,DANGEROUS DRUGS,511.0,"CONTROLLED SUBSTANCE, POSSESSI",...,BUS TERMINAL,NaN,NaN,986882.0,214802.0,40.756266,-73.990501,"(40.756266207, -73.990501248)","(-73.990501248, 40.756266207)",POINT (-73.990501248 40.756266207)
1,155423129,12/31/2016,23:40:00,12/31/2016,23:50:00,12/31/2016,105,ROBBERY,389.0,"ROBBERY,DWELLING",...,RESIDENCE - APT. HOUSE,NaN,NaN,1021170.0,241240.0,40.828755,-73.866594,"(40.828754623, -73.866593516)","(-73.866593516, 40.828754623)",POINT (-73.86659351600001 40.828754623)
2,653964645,12/31/2016,23:30:00,12/31/2016,23:31:00,12/31/2016,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,STREET,NaN,NaN,1001512.0,234334.0,40.809860,-73.937644,"(40.809859893, -73.937644103)","(-73.937644103, 40.809859893)",POINT (-73.937644103 40.809859893)
3,988275798,12/31/2016,23:25:00,NaN,NaN,12/31/2016,235,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",...,STREET,NaN,NaN,987182.0,201484.0,40.719711,-73.989424,"(40.719711494, -73.9894242)","(-73.9894242, 40.719711494)",POINT (-73.9894242 40.719711494)
4,225104473,12/31/2016,23:24:00,12/31/2016,23:30:00,12/31/2016,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",...,STREET,NaN,NaN,1026086.0,192340.0,40.694515,-73.849134,"(40.694514975, -73.849134227)","(-73.849134227, 40.694514975)",POINT (-73.84913422699999 40.694514975)


In [ ]:
from choroplethNYC import choroplethNYC

fig, ax = choroplethNYC.choroplethNYC(df)
noisegdf.plot(ax=ax)

In [ ]:
f, ax, cb = cp.choroplethNYC(gdf, "column_name", kind='continuous', cmap="viridis")

In [75]:
df.BORO_NM

0             MANHATTAN
1                 BRONX
2             MANHATTAN
3             MANHATTAN
4                QUEENS
5              BROOKLYN
6              BROOKLYN
7             MANHATTAN
8              BROOKLYN
9                QUEENS
10             BROOKLYN
11             BROOKLYN
12            MANHATTAN
13            MANHATTAN
14            MANHATTAN
15                BRONX
16             BROOKLYN
17            MANHATTAN
18               QUEENS
19            MANHATTAN
20                BRONX
21        STATEN ISLAND
22                BRONX
23            MANHATTAN
24             BROOKLYN
25            MANHATTAN
26                BRONX
27                BRONX
28                BRONX
29             BROOKLYN
              ...      
478773           QUEENS
478774           QUEENS
478775           QUEENS
478776            BRONX
478777           QUEENS
478778         BROOKLYN
478779           QUEENS
478780           QUEENS
478781           QUEENS
478782         BROOKLYN
478783          